# Upshot:
- I searched through the following corpora to see what kinds of transcript data and demographic details of the speakers (age, sex, etc) were available
    - 

In [1]:
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
import re
from pathlib import Path
from tqdm import tqdm